## Importing the relevant libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [5]:
file = open("ref_txt.txt", "r", encoding = "utf8")
lines = []

for i in file:
    lines.append(i)

lines[10]

'a proper human room although a little too small, lay peacefully\n'

## Cleaning the data

In [7]:
data = ""

for i in lines:
    data = ' '. join(lines)
    
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')

data[:200]

'One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin.  He lay on his armour-like back, and if he lifted his head a little he could s'

In [8]:
import string

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
new_data = data.translate(translator)

new_data[:200]


'One morning  when Gregor Samsa woke from troubled dreams  he found himself transformed in his bed into a horrible vermin   He lay on his armour like back  and if he lifted his head a little he could s'

In [9]:
z = []

for i in data.split():
    if i not in z:
        z.append(i)
        
data = ' '.join(z)
data[:200]

'One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on armour-like back, and if lifted head little could see brown belly, s'

## Tokenization

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function.
pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:10]

[17, 53, 293, 2, 18, 729, 135, 730, 294, 8]

In [11]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

2617


In [12]:
sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  3889


array([[ 17,  53],
       [ 53, 293],
       [293,   2],
       [  2,  18],
       [ 18, 729],
       [729, 135],
       [135, 730],
       [730, 294],
       [294,   8],
       [  8, 731]])

In [13]:
X = []
y = []

for i in sequences:
    X.append(i[0])
    y.append(i[1])
    
X = np.array(X)
y = np.array(y)

In [14]:
print("The Data is: ", X[:5])
print("The responses are: ", y[:5])

The Data is:  [ 17  53 293   2  18]
The responses are:  [ 53 293   2  18 729]


In [15]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## Creating the model

In [16]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 10)             26170     
                                                                 
 lstm (LSTM)                 (None, 1, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 2617)              2619617   
                                                                 
Total params: 15,694,787
Trainable params: 15,694,787
Non-trainable params: 0
_________________________________________________________________


## Callbacks

In [21]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint("nextword1.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

## Compiling and fitting the model

In [24]:
model.compile(loss="categorical_crossentropy", optimizer='adam')
model.fit(X, y, epochs=150, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])

Epoch 1/150
61/61 [==============================] - ETA: 0s - loss: 7.2529
Epoch 1: loss improved from 7.33149 to 7.25295, saving model to nextword1.h5
61/61 [==============================] - 21s 172ms/step - loss: 7.2529 - lr: 0.0010
Epoch 2/150
61/61 [==============================] - ETA: 0s - loss: 7.0991
Epoch 2: loss improved from 7.25295 to 7.09909, saving model to nextword1.h5
61/61 [==============================] - 10s 167ms/step - loss: 7.0991 - lr: 0.0010
Epoch 3/150
61/61 [==============================] - ETA: 0s - loss: 7.0114
Epoch 3: loss improved from 7.09909 to 7.01141, saving model to nextword1.h5
61/61 [==============================] - 11s 182ms/step - loss: 7.0114 - lr: 0.0010
Epoch 4/150
61/61 [==============================] - ETA: 0s - loss: 6.9464
Epoch 4: loss improved from 7.01141 to 6.94640, saving model to nextword1.h5
61/61 [==============================] - 11s 181ms/step - loss: 6.9464 - lr: 0.0010
Epoch 5/150
61/61 [==============================] -

61/61 [==============================] - 8s 129ms/step - loss: 3.7383 - lr: 0.0010
Epoch 36/150
61/61 [==============================] - ETA: 0s - loss: 3.6262
Epoch 36: loss improved from 3.73831 to 3.62621, saving model to nextword1.h5
61/61 [==============================] - 8s 130ms/step - loss: 3.6262 - lr: 0.0010
Epoch 37/150
61/61 [==============================] - ETA: 0s - loss: 3.4764
Epoch 37: loss improved from 3.62621 to 3.47641, saving model to nextword1.h5
61/61 [==============================] - 8s 131ms/step - loss: 3.4764 - lr: 0.0010
Epoch 38/150
61/61 [==============================] - ETA: 0s - loss: 3.3688
Epoch 38: loss improved from 3.47641 to 3.36882, saving model to nextword1.h5
61/61 [==============================] - 8s 133ms/step - loss: 3.3688 - lr: 0.0010
Epoch 39/150
61/61 [==============================] - ETA: 0s - loss: 3.3109
Epoch 39: loss improved from 3.36882 to 3.31089, saving model to nextword1.h5
61/61 [==============================] - 8s 129m

61/61 [==============================] - ETA: 0s - loss: 1.8951
Epoch 70: loss improved from 1.90393 to 1.89512, saving model to nextword1.h5
61/61 [==============================] - 9s 153ms/step - loss: 1.8951 - lr: 0.0010
Epoch 71/150
61/61 [==============================] - ETA: 0s - loss: 1.8692
Epoch 71: loss improved from 1.89512 to 1.86920, saving model to nextword1.h5
61/61 [==============================] - 10s 163ms/step - loss: 1.8692 - lr: 0.0010
Epoch 72/150
61/61 [==============================] - ETA: 0s - loss: 1.8448
Epoch 72: loss improved from 1.86920 to 1.84483, saving model to nextword1.h5
61/61 [==============================] - 10s 158ms/step - loss: 1.8448 - lr: 0.0010
Epoch 73/150
61/61 [==============================] - ETA: 0s - loss: 1.8354
Epoch 73: loss improved from 1.84483 to 1.83544, saving model to nextword1.h5
61/61 [==============================] - 10s 156ms/step - loss: 1.8354 - lr: 0.0010
Epoch 74/150
61/61 [==============================] - ETA:

Epoch 105/150
61/61 [==============================] - ETA: 0s - loss: 0.8402
Epoch 105: loss improved from 0.84599 to 0.84016, saving model to nextword1.h5
61/61 [==============================] - 9s 155ms/step - loss: 0.8402 - lr: 2.0000e-04
Epoch 106/150
61/61 [==============================] - ETA: 0s - loss: 0.8354
Epoch 106: loss improved from 0.84016 to 0.83540, saving model to nextword1.h5
61/61 [==============================] - 9s 152ms/step - loss: 0.8354 - lr: 2.0000e-04
Epoch 107/150
61/61 [==============================] - ETA: 0s - loss: 0.8300
Epoch 107: loss improved from 0.83540 to 0.83001, saving model to nextword1.h5
61/61 [==============================] - 10s 158ms/step - loss: 0.8300 - lr: 2.0000e-04
Epoch 108/150
61/61 [==============================] - ETA: 0s - loss: 0.8250
Epoch 108: loss improved from 0.83001 to 0.82505, saving model to nextword1.h5
61/61 [==============================] - 10s 157ms/step - loss: 0.8250 - lr: 2.0000e-04
Epoch 109/150
61/61 [=

Epoch 140/150
61/61 [==============================] - ETA: 0s - loss: 0.7538
Epoch 140: loss improved from 0.75869 to 0.75381, saving model to nextword1.h5
61/61 [==============================] - 9s 155ms/step - loss: 0.7538 - lr: 2.0000e-04
Epoch 141/150
61/61 [==============================] - ETA: 0s - loss: 0.7547
Epoch 141: loss did not improve from 0.75381
61/61 [==============================] - 9s 148ms/step - loss: 0.7547 - lr: 2.0000e-04
Epoch 142/150
61/61 [==============================] - ETA: 0s - loss: 0.7546
Epoch 142: loss did not improve from 0.75381
61/61 [==============================] - 9s 146ms/step - loss: 0.7546 - lr: 2.0000e-04
Epoch 143/150
61/61 [==============================] - ETA: 0s - loss: 0.7511
Epoch 143: loss improved from 0.75381 to 0.75106, saving model to nextword1.h5
61/61 [==============================] - 9s 153ms/step - loss: 0.7511 - lr: 2.0000e-04
Epoch 144/150
61/61 [==============================] - ETA: 0s - loss: 0.7498
Epoch 144: los

## Summary

The loss significantly improved from 7.33 to 0.74 over 150 epochs

The learning rate becomes negligible after epoch 97. Which means the model has reached it's maximum capacity at 97th epoch itself

As there is a steady decline in loss, this is a good model